In [26]:
import requests
import json
import pandas as pd
from datetime import datetime
from tqdm import tqdm

In [24]:
t1 = datetime.now()
t2 = datetime.now()

0:00:00.000077


In [17]:
rules = pd.read_json('rules.json').sample(1000,random_state=20221106).reset_index(drop=True)

In [18]:
rules.sort_values('confidence', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
944,"[gaseosa, postre_individual, helado]",[map],0.002736,0.359848,0.002531,0.925234,2.571181,0.001547,8.562036
522,"[gaseosa, barquillo, postre_individual]",[map],0.000928,0.359848,0.000800,0.862205,2.396026,0.000466,4.645676
576,"[agua, postre_individual]",[map],0.004197,0.359848,0.003601,0.858138,2.384724,0.002091,4.512484
638,"[queso_laminado, mantequilla, bolleria]",[panaderia],0.000858,0.368446,0.000683,0.795745,2.159734,0.000367,3.091985
908,"[queso_laminado, bolleria, jamon]",[panaderia],0.003685,0.368446,0.002864,0.777007,2.108878,0.001506,2.832170
...,...,...,...,...,...,...,...,...,...
571,[map],"[plato_liviano, jamon]",0.359848,0.001366,0.000511,0.001421,1.040250,0.000020,1.000055
407,[map],"[empanada_individual, postre_individual, sopas]",0.359848,0.001023,0.000508,0.001411,1.379552,0.000140,1.000389
619,[panaderia],"[queso_laminado, jamon, map]",0.368446,0.000902,0.000511,0.001388,1.538359,0.000179,1.000486
170,[panaderia],"[charcuteria, salame]",0.368446,0.001183,0.000508,0.001378,1.164384,0.000072,1.000195


In [28]:
times = []
for row,column in tqdm(rules.iterrows()):
    
    basket = column['antecedents']
    basket_string = ','.join(basket)
    
    url_api_rquest = f'https://us-central1-mds-recsys.cloudfunctions.net/forkrecsys2?basket={basket_string}'
    
    t1 = datetime.now()
    r = requests.post(url_api_rquest)
    t2 = datetime.now()
    
    dict_storage = {'basket': basket, 'api_response': r.json(), 'time_response':t2-t1}
    times.append(dict_storage)

1000it [08:29,  1.96it/s]


In [11]:
r = requests.post(f'https://us-central1-mds-recsys.cloudfunctions.net/forkrecsys2?basket=gaseosa,postre_individual,helado')
r.json()

{'rec_0': 'map', 'rec_1': 'jugo', 'rec_2': 'panaderia'}

In [35]:
df_times = pd.DataFrame(times)
df_times['total_seconds'] = df_times['time_response'].apply(lambda x: x.total_seconds())

In [43]:
df_times.to_csv('resultados/tiempo_consulta_api.csv')
#df_times = pd.read_json('resultados/tiempo_consulta_api.json')

In [42]:
df_times[['total_seconds']].describe()

,total_seconds
count,1000.000000
mean,0.507053
std,0.100637
min,0.342358
25%,0.410037
50%,0.508622
75%,0.514407
max,0.923693


In [41]:
import numpy as np
np.percentile(df_times.total_seconds, 90)

0.6163606